<a href="https://colab.research.google.com/github/wmrevel/IA-LABS/blob/main/content/LAB%2002.03%20-%20PANDAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB 02.03 - Pandas

In [26]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()


In [38]:
from local.lib.rlxmoocapi import submit, session
session.LoginSequence(endpoint=init.endpoint, course_id=init.course_id, lab_id="L02.03", varname="student");

logging in as wmauricio.restrepo@udea.edu.co... please wait
using course session ai4eng.v1.udea.r2::2023.1
success!! you are logged in


In [7]:
import numpy as np
import pandas as pd

In [113]:
a=np.random.rand(6)
b=np.random.randint(1,10,6)  #random.randint(19,65,10)
print(np.allclose(a,b))
print(np.any(a==b))
mean=np.mean(b)
print(a)
print(b)
print(b[b<5])
print(mean)
c=a.reshape(2,-1)
c.flatten()
print(c)


False
False
[0.75122227 0.76562699 0.75139442 0.08804653 0.66764107 0.66583463]
[1 4 7 1 3 2]
[1 4 1 3 2]
3.0
[[0.75122227 0.76562699 0.75139442]
 [0.08804653 0.66764107 0.66583463]]


In [6]:
e = np.full((2,3),0)
e

array([[0, 0, 0],
       [0, 0, 0]])

## Task 1: Extract data

we have a dataframe of items with a price and, **sometimes**, with an extra column (`margin`).

You will have to complete a function that will **filter** the dataframe selecting the rows:

- whose price is > 100
- **OR** whose margin is >10, if the column `margin` is present

your function must return **A LIST** with the item ids of the selected rows

For instance, with the following dataframe

             price  category
    itemid                  
    56556   108.15       2.0
    73065    83.54       2.0
    36619   114.42       0.0
    73414    82.94       0.0
    13410   115.13       2.0
    66153    91.50       1.0
    77380    85.82       0.0
    73249    95.95       0.0
    11654   100.22       2.0
    11972    77.16       1.0

your must return the following list:

     [56556, 36619, 13410, 11654]
     
But if you get the following dataframe

             price  category  margin
    itemid                          
    39059    98.11       0.0   11.04
    19526    98.11       1.0   11.25
    78176    94.34       1.0   10.51
    50948   102.37       1.0   10.77
    12111    98.07       1.0    8.50
    56191    98.53       1.0   11.65
    38887    91.49       2.0   11.24
    77915   117.30       0.0    8.64
    55010    96.13       0.0    8.95
    45925    98.59       1.0   10.45
    
you must return the following list

    [39059, 19526, 78176, 50948, 56191, 38887, 77915, 45925]

In [6]:
def create_df(missing=False, n=10):
    itemid   = np.random.randint(100000, size=n)+1000
    category = np.random.randint(3, size=n)
    price    = np.round(np.random.normal(loc=100, scale=10, size=n),2)
    margin   = np.round(np.random.normal(loc=10, scale=1, size=n),2)
    
    if missing:
        nmissing = np.random.randint(len(price)//2)+2                                     
        price[np.random.permutation(len(price))[:nmissing]] = np.nan
    
    d = pd.DataFrame(np.r_[[price, category, margin]].T, index=itemid, columns=["price", "category", "margin"])
    d.index.name="itemid"
    if np.random.random()>.5:
        d = d[d.columns[:2]]
    #print(d[d.columns[:2]])
    
    return d

In [312]:
d = create_df()
d.shape

(10, 2)

In [8]:
def select_items(df):
    # make sure to make a copy in case you modify the original df
    df = df.copy()
    #print(len(df),df.shape[1])
    if df.shape[1]==3:
      r = df[(df.price>100)|((df.margin>10))]
    else:
      r = df[(df.price>100)]
      
    #... # YOUR CODE HERE
    #print(p100)
    result = r.index.tolist()
    return result

**manually check your answer**

In [9]:
d = create_df()
d

,price,category
itemid,,
59823,85.32,2.0
60997,124.44,2.0
22626,98.24,0.0
33491,89.16,2.0
99436,86.55,2.0
88610,103.55,1.0
3141,110.66,1.0
53762,98.02,1.0
33009,103.61,2.0


In [10]:
select_items(d)

[60997, 88610, 3141, 33009]

**submit your code**

In [346]:
student.submit_task(globals(), task_id="task_01");

## Task 2: Group statistics

Complete the following function so that it returns a dataframe with the average, max and min **prices** per category. 

For instance, for the following dataframe

             price  category  margin
    itemid                          
    17946    93.85       1.0   10.64
    61190    91.72       1.0    9.76
    39639   100.16       1.0   10.67
    17791   110.44       2.0    9.65
    7333    101.05       1.0    9.69
    77362   122.33       0.0   11.14
    92646   108.13       2.0   10.58
    27797    85.52       2.0   10.88
    31746    97.56       0.0    9.75
    12355   101.04       2.0    9.51
    
you should return the following dataframe

                 media  maximo  minimo
    categoria                         
    0         109.9450  122.33  97.56
    1          96.6950  101.05  91.72
    2         101.2825  110.44  85.52
    
observe that your result
- must not be a **multilevel** columnset.
- the column names and the index name must be **exactly** as in the example.
- the **index** must be of type **int**.

In [67]:
def get_stats(df):
    # make sure to make a copy in case you modify the original df
    df = df.copy()
    
    #... # YOUR CODE HERE    
    result = df.groupby('category')['price'].agg(['mean', 'max', 'min'])
    result.columns=["media", "maximo", "minimo"]
    result[0] = result[0].astype(int)
    result.index.name="categoria"
    result.head()
    
    return result

**manually check your answer**

In [68]:
d = create_df()
d

,price,category,margin
itemid,,,
24545,116.69,2.0,9.59
48920,109.98,0.0,10.33
14250,88.29,1.0,12.19
60534,97.69,1.0,8.44
14412,108.09,1.0,9.50
38083,105.45,2.0,9.52
32830,101.25,1.0,11.63
24444,104.86,2.0,10.11
69245,81.18,1.0,9.52


In [69]:
get_stats(d)

KeyError: ignored

**submit your code**

In [39]:
student.submit_task(globals(), task_id="task_02");

## Task 3: Fill in missing data

Fill in the missing data in the **price** column with the following procedure:

- compute the mean and std of the avaialable prices
- sample from a normal distribution with the computed mean and std (see [`np.random.normal`])(https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html), as many samples as missing data
- substitute the missing values with the samples

For instance, for this input dataframe:

             price  category  margin
    itemid                          
    18922      NaN       1.0   10.32
    69500   121.25       1.0   10.22
    76442    90.25       1.0   12.60
    33863   106.51       0.0   10.26
    15904    95.87       1.0   11.51
    41946   103.47       2.0    9.85
    85451    93.08       2.0    9.56
    70028   116.68       1.0    9.11
    26860      NaN       2.0    9.71
    12807    91.48       0.0    9.77
    
your solution might be similar to this (not exactly the same as you will be sampling data):


                 price  category  margin
    itemid                              
    18922    97.441188       1.0   10.32
    69500   121.250000       1.0   10.22
    76442    90.250000       1.0   12.60
    33863   106.510000       0.0   10.26
    15904    95.870000       1.0   11.51
    41946   103.470000       2.0    9.85
    85451    93.080000       2.0    9.56
    70028   116.680000       1.0    9.11
    26860   103.294843       2.0    9.71
    12807    91.480000       0.0    9.77
    
    
**WARN**: your function must not modify the original dataframe, make a copy of the input dataframe, fill the values in the copy and return it.

**HINT**: use the [`isna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isna.html) method of a dataframe or a series.

In [ ]:
d = create_df(missing=True)
d

In [ ]:
def fillna(df):
    # make sure to make a copy in case you modify the original df
    df = df.copy()
    
    ... # YOUR CODE HERE    
    result = ...
    return result

**check manually your code**

In [ ]:
d = create_df(missing=True)
d

In [ ]:
fillna(d)

**submit your code**

In [ ]:
student.submit_task(globals(), task_id="task_03");